In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Operacionalización de datos

### Alumno desertor
- Requisito: No se debe haber titulado para la carrera analizada

### Tipos de deserción

- Sale del sistema y no vuelve a ingresar
- Sale del sistema y vuelve después de 1, 2 ó 3 años y termina o sigue en el sistema (seudo deserción?):
    - Vuelve a la misma carrera en la misma institución (no deserta, "congela")
    - Vuelve a otra carrera en la misma institución
    - Vuelve a la misma carrera en otra institución
    - Vuelve a otra carrera en otra institución
- Sale del sistema, vuelve a ingresar, pero no termina (múltitple deserción?)

# Matriculados

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Se considera todos los matriculados en CFT / IP desde 2015

In [2]:
# Matriculados de CFT / IP
mat_cft_ip = pd.read_csv(f"{BASE}/tmp_data/fechas_titulacion.csv")
mat_cft_ip.head()

,codigo_unico,mrun,fecha_obtencion_titulo
0,I143S26C9J2V2,26379527.0,20220228.0
1,I260S22C108J1V1,4120917.0,20220228.0
2,I116S34C17J2V1,4348827.0,20220228.0
3,I260S21C263J2V1,20301532.0,20220228.0
4,I143S2C651J2V1,3498846.0,20220228.0


In [3]:
mat_cft_ip.isna().sum()

codigo_unico                    0
mrun                            0
fecha_obtencion_titulo    1194617
dtype: int64

In [4]:
# Se carga datos de matrícula para estos alumnos, considerando todas las instituciones
cols = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 'dur_estudio_carr',
    'cod_inst', 'cod_carrera'
]

mruns = mat_cft_ip["mrun"].unique()

matriculados = pd.concat([
    iter_df_read(f, "mrun", mruns, sep=";", usecols=cols)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# Se agrega columna que indica titulación

if "fecha_obtencion_titulo" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        mat_cft_ip,
        how="left"
    )
    
    assert matriculados.shape[0] == len_or

## Pre procesamiento: Año de titulación

In [6]:
matriculados["anio_titulacion"] = matriculados["fecha_obtencion_titulo"].fillna("").astype(str).str[:4].replace([""], [np.nan]).astype(float)
matriculados["anio_titulacion"].describe().round()

count    1073280.0
mean        2019.0
std            1.0
min         2015.0
25%         2018.0
50%         2020.0
75%         2021.0
max         2022.0
Name: anio_titulacion, dtype: float64

## Pre procesamiento: Último año de matrícula por carrera

In [7]:
if "ultimo_anio_matricula" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        matriculados.groupby(["mrun", "codigo_unico"]).agg({
            "cat_periodo": "max"
        }).reset_index().rename(columns={
            "cat_periodo": "ultimo_anio_matricula"
        }),
        how="left"
    ).drop_duplicates().reset_index(drop=True)
    
    assert matriculados.shape[0] == len_or

In [8]:
matriculados["ultimo_anio_matricula"].describe().round()

count    3743349.0
mean        2020.0
std            2.0
min         2015.0
25%         2018.0
50%         2020.0
75%         2022.0
max         2022.0
Name: ultimo_anio_matricula, dtype: float64

## Pre prcesamiento: Deserciones

### Primer caso: Sale del sistema y no vuelve a ingresar
- Solo han tomado 1 carrera
- No se han titulado
- Puede o no tener "años en blanco"
- No se han matriculado el año 2022

In [9]:
cft_ip = ["Institutos Profesionales", "Centros de Formación Técnica"]

In [10]:
ruts_matriculados_cft_ip = matriculados[
    matriculados["tipo_inst_1"].isin(cft_ip)
]["mrun"].unique()

print("Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total): ", ruts_matriculados_cft_ip.shape[0])

Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total):  1233136


In [11]:
cant_carreras = matriculados[
    matriculados["mrun"].isin(ruts_matriculados_cft_ip)
][["mrun", "codigo_unico"]].drop_duplicates().groupby(["mrun"]).size().reset_index(name="cant_carreras")

ruts_1_carrera = cant_carreras[cant_carreras["cant_carreras"] == 1]["mrun"].unique()

print("Dentro de los anteriores, cantidad que solo ha tomado 1 carrera: ", ruts_1_carrera.shape[0])

Dentro de los anteriores, cantidad que solo ha tomado 1 carrera:  741611


In [12]:
ruts_desertores_1 = matriculados[
    (matriculados["mrun"].isin(ruts_1_carrera))
    & (matriculados["anio_titulacion"].isna())
    & (matriculados["ultimo_anio_matricula"] < 2022)
]["mrun"].unique()

print("Dentro de los anteriores, no se han titulado ni matriculado el año 2022: ", ruts_desertores_1.shape[0])

Dentro de los anteriores, no se han titulado ni matriculado el año 2022:  349677


In [13]:
print(f"% Desertores Criterio 1: {np.round(ruts_desertores_1.shape[0] / ruts_matriculados_cft_ip.shape[0] * 100, 1)}%")

% Desertores Criterio 1: 28.4%


### Segundo caso:  Sale del sistema y vuelve a la misma carrera en la misma institución
- Dentro de una misma carrera, tiene años en blanco (1, 2 ó 3)
- En los años en blanco, no se ha matriculado en otra carrera
- Se titula o se matricula en el año 2022 (no desertó completamente)

In [14]:
max_dif_entre_mat_rut_carr = matriculados[
    (matriculados["mrun"].isin(ruts_matriculados_cft_ip))
    & (
        (matriculados["anio_titulacion"].notna())
        | (matriculados["ultimo_anio_matricula"] == 2022)
    )
].groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": get_array_diff
}).reset_index().rename(columns={
    "cat_periodo": "anios_en_blanco"
})

max_dif_entre_mat_rut_carr.head()

,mrun,codigo_unico,anios_en_blanco
0,19.0,I536S0C39J1V1,1
1,98.0,I111S22C8J1V1,1
2,99.0,I111S12C420J1V1,1
3,118.0,I221S1C113J2V1,1
4,196.0,I280S8C380J1V1,1


In [22]:
t = matriculados[
    (matriculados["mrun"].isin(ruts_matriculados_cft_ip))
    & (
        (matriculados["anio_titulacion"].notna())
        | (matriculados["ultimo_anio_matricula"] == 2022)
    )
].groupby(["mrun", "codigo_unico"]).agg(
    min_anio=("cat_periodo", "min"),
    max_anio=("cat_periodo", "max"),
    max_dif_anios=("cat_periodo", get_array_diff)
).reset_index()

t.head()

,mrun,codigo_unico,min_anio,max_anio,max_dif_anios
0,19.0,I536S0C39J1V1,2019,2021,1
1,98.0,I111S22C8J1V1,2021,2022,1
2,99.0,I111S12C420J1V1,2021,2022,1
3,118.0,I221S1C113J2V1,2016,2017,1
4,196.0,I280S8C380J1V1,2017,2019,1


In [26]:
t2 = t[
    t["mrun"].isin(
        t[t["max_dif_anios"] > 1]["mrun"].unique() # ruts con años en blanco
    )
].groupby(["mrun"]).size().reset_index(name="cant_carreras")

t2.head()

,mrun,cant_carreras
0,1118.0,1
1,1143.0,1
2,2299.0,1
3,2896.0,1
4,3266.0,1


In [35]:
t3 = t[
    t["mrun"].isin(
        t2[t2["cant_carreras"] > 1]["mrun"].unique() # ruts con años en blanco y con más de una carrera
    )
]

t3["tiene_anios_en_blanco"] = np.where(t3["max_dif_anios"] > 1, 1, 0)
t3.head()

/tmp/ipykernel_13428/2851122300.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t3["tiene_anios_en_blanco"] = np.where(t3["max_dif_anios"] > 1, 1, 0)


,mrun,codigo_unico,min_anio,max_anio,max_dif_anios,tiene_anios_en_blanco
237,7778.0,I111S3C48J1V1,2015,2018,1,0
238,7778.0,I75S1C113J1V1,2020,2022,2,1
273,8652.0,I22S1C1J2V3,2020,2022,2,1
274,8652.0,I260S38C108J1V1,2015,2017,1,0
291,9124.0,I111S20C413J2V1,2019,2022,3,1


In [88]:
def tiene_carrera_entre_medio(data, mrun, codigo_unico):
    try:
        max_min_otra_c = data[
            (data["mrun"]==mrun) & (data["codigo_unico"]!=codigo_unico)
        ].groupby(["mrun"]).agg({"min_anio": "max"})["min_anio"].iloc[0]

        min_max_otra_c = data[
            (data["mrun"]==mrun) & (data["codigo_unico"]!=codigo_unico)
        ].groupby(["mrun"]).agg({"max_anio": "min"})["max_anio"].iloc[0]

        tiene_carrera_entre_medio = 0

        con_anios_blanco = data[
            (data["mrun"]==mrun) & (data["codigo_unico"]==codigo_unico)
        ]
        #print(mrun, codigo_unico, max_min_otra_c, min_max_otra_c)
        for _, r in con_anios_blanco.iterrows():
         #   print(r["min_anio"], r["max_anio"])
            if max_min_otra_c < r["max_anio"] and min_max_otra_c > r["min_anio"]:
                tiene_carrera_entre_medio = 1
                break

        return tiene_carrera_entre_medio
    except Exception as e:
        print(e)
        print(mrun)
        raise(e)

tiene_anios_en_blanco = t3[t3["tiene_anios_en_blanco"]==1]
tiene_anios_en_blanco["tiene_carrera_entre_medio"] = [
    tiene_carrera_entre_medio(t3, m, c)
    for m, c in tqdm(
        zip(tiene_anios_en_blanco["mrun"], tiene_anios_en_blanco["codigo_unico"]),
        total=tiene_anios_en_blanco.shape[0]
    )
]

  0%|          | 0/3610 [00:00<?, ?it/s]

/tmp/ipykernel_13428/2603680547.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiene_anios_en_blanco["tiene_carrera_entre_medio"] = [


In [96]:
tiene_anios_en_blanco_sin_otra_carrera = tiene_anios_en_blanco[tiene_anios_en_blanco["tiene_carrera_entre_medio"]==0]
tiene_anios_en_blanco_sin_otra_carrera.head()

,mrun,codigo_unico,min_anio,max_anio,max_dif_anios,tiene_anios_en_blanco,tiene_carrera_entre_medio
238,7778.0,I75S1C113J1V1,2020,2022,2,1,0
273,8652.0,I22S1C1J2V3,2020,2022,2,1,0
291,9124.0,I111S20C413J2V1,2019,2022,3,1,0
812,25445.0,I117S1C27J4V2,2016,2019,3,1,0
822,25742.0,I19S3C21J1V1,2019,2022,3,1,0


In [98]:
ruts_desertores_2_1 = tiene_anios_en_blanco_sin_otra_carrera[tiene_anios_en_blanco_sin_otra_carrera["max_dif_anios"] == 2][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_2 = tiene_anios_en_blanco_sin_otra_carrera[tiene_anios_en_blanco_sin_otra_carrera["max_dif_anios"] == 3][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_3 = tiene_anios_en_blanco_sin_otra_carrera[tiene_anios_en_blanco_sin_otra_carrera["max_dif_anios"] == 4][["mrun", "codigo_unico"]].drop_duplicates()

print(f"Cant. matriculas de CFT/IP que congelaron 1 año: ", ruts_desertores_2_1.shape[0])
print(f"Cant. matriculas de CFT/IP que congelaron 2 años: ", ruts_desertores_2_2.shape[0])
print(f"Cant. matriculas de CFT/IP que congelaron 3 años: ", ruts_desertores_2_3.shape[0])

Cant. matriculas de CFT/IP que congelaron 1 año:  2635
Cant. matriculas de CFT/IP que congelaron 2 años:  436
Cant. matriculas de CFT/IP que congelaron 3 años:  110


In [101]:
print(f"% Desertores Criterio 2.1: {np.round(ruts_desertores_2_1.shape[0] / ruts_matriculados_cft_ip.shape[0] * 100, 2)}%")
print(f"% Desertores Criterio 2.2: {np.round(ruts_desertores_2_2.shape[0] / ruts_matriculados_cft_ip.shape[0] * 100, 2)}%")
print(f"% Desertores Criterio 2.2: {np.round(ruts_desertores_2_3.shape[0] / ruts_matriculados_cft_ip.shape[0] * 100, 2)}%")

% Desertores Criterio 2.1: 0.21%
% Desertores Criterio 2.2: 0.04%
% Desertores Criterio 2.2: 0.01%


### Tercer caso: Vuelve a otra carrera en la misma institución
- No se han titulado de su primera carrera
- Sin "años en banco"
- Se titula de su enésima carrera o se ha matriculado en el año 2022 (no desertó completamente)

In [ ]:
matriculados[
    (matriculados["mrun"].isin(ruts_matriculados_cft_ip))
    & (
        (matriculados["anio_titulacion"].notna())
        | (matriculados["ultimo_anio_matricula"] == 2022)
    )
].groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": get_array_diff
}).reset_index().rename(columns={
    "cat_periodo": "anios_en_blanco"
})